Modules communicate using pulses. Each pulse is either a high pulse or a low pulse. When a module sends a pulse, it sends that type of pulse to each module in its list of destination modules.

There are several different types of modules:

Flip-flop modules (prefix %) are either on or off; they are initially off. If a flip-flop module receives a high pulse, it is ignored and nothing happens. However, if a flip-flop module receives a low pulse, it flips between on and off. If it was off, it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.

Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; they initially default to remembering a low pulse for each input. When a pulse is received, the conjunction module first updates its memory for that input. Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.

There is a single broadcast module (named broadcaster). When it receives a pulse, it sends the same pulse to all of its destination modules.

Here at Desert Machine Headquarters, there is a module with a single button on it called, aptly, the button module. When you push the button, a single low pulse is sent directly to the broadcaster module.

After pushing the button, you must wait until all pulses have been delivered and fully handled before pushing it again. Never push the button if modules are still processing pulses.

Pulses are always processed in the order they are sent. So, if a pulse is sent to modules a, b, and c, and then module a processes its pulse and sends more pulses, the pulses sent to modules b and c would have to be handled first.

The module configuration (your puzzle input) lists each module. The name of the module is preceded by a symbol identifying its type, if any. The name is then followed by an arrow and a list of its destination modules.

Consult your module configuration; determine the number of low pulses and high pulses that would be sent after pushing the button 1000 times, waiting for all pulses to be fully handled after each push of the button. What do you get if you multiply the total number of low pulses sent by the total number of high pulses sent?

In [ ]:
mods = {}
results = {0: 0,
           1: 0}

conjunctions = []
with open('input.txt') as file:
    for line in file:
        mod, outs = line.strip().split(' -> ')
        outs = outs.split(', ')
        if mod != 'broadcaster':
            # modules[label] : [pf:(% or &), outputs:[...], status or mem: (0/1, or {})
            pf = mod[0]
            label = mod[1:]
            if pf == '%':
                mods[label] = [pf, outs, 0]
            else:
                mods[label] = [pf, outs]
                conjunctions.append(label)
        else:
            mods['broadcaster'] = outs

for c in conjunctions:
    opt_dict = {}
    for k, v in mods.items():
        if k != 'broadcaster':
            if c in v[1]:
                opt_dict[k] = 0
    mods[c].append(opt_dict)

queue = []
def button_press(pulse, label, input):   
    p_cnt = 0
    p = 0
    
    if label == 'broadcaster':
        for m in mods[label]:
            p_cnt += 1
            queue.append((pulse, m, label))
    
        return pulse, p_cnt
    
    elif label in mods:
        pf, outputs, opt = mods[label]  
        if pf == '%':
            if pulse == 0:
                mods[label][2] = not opt
                p = mods[label][2]
            else:
                outputs = []
        elif pf == '&':
            opt[input] = pulse
            p = (0 if sum(opt.values()) == len(opt) else 1)
        
        for m in outputs:
            queue.append((p, m, label))
            p_cnt += 1
        
        return p, p_cnt
    
    else:
        return 0, 0

for i in range(1000):
    queue.append((0, 'broadcaster', None))
    results[0] += 1
    while queue:
        p, l, i = queue.pop(0)
        k, c = button_press(p, l, i)
        results[k] += c

product = 1
for val in results.values():
    product *= val

print(product)

The final machine responsible for moving the sand down to Island Island has a module attached named rx. The machine turns on when a single low pulse is sent to rx.

Reset all modules to their default states. Waiting for all pulses to be fully handled after each button press, what is the fewest number of button presses required to deliver a single low pulse to the module named rx?

In [1]:
import math
from collections import deque

class Module:
    def __init__(self, name, type, outputs):
        self.name = name
        self.type = type
        self.outputs = outputs

        if type == "%":
            self.memory = "off"
        else:
            self.memory = {}
    def __repr__(self):
        return self.name + "{type=" + self.type + ",outputs=" + ",".join(self.outputs) + ",memory=" + str(self.memory) + "}"

modules = {}
broadcast_targets = []

for line in open('input.txt'):
    left, right = line.strip().split(" -> ")
    outputs = right.split(", ")
    if left == "broadcaster":
        broadcast_targets = outputs
    else:
        type = left[0]
        name = left[1:]
        modules[name] = Module(name, type, outputs)

for name, module in modules.items():
    for output in module.outputs:
        if output in modules and modules[output].type == "&":
            modules[output].memory[name] = "lo"

(feed,) = [name for name, module in modules.items() if "rx" in module.outputs]

cycle_lengths = {}
seen = {name: 0 for name, module in modules.items() if feed in module.outputs}

presses = 0

while True:
    presses += 1
    q = deque([("broadcaster", x, "lo") for x in broadcast_targets])
    
    while q:
        origin, target, pulse = q.popleft()
        
        if target not in modules:
            continue
        
        module = modules[target]
        
        if module.name == feed and pulse == "hi":
            seen[origin] += 1

            if origin not in cycle_lengths:
                cycle_lengths[origin] = presses
            else:
                assert presses == seen[origin] * cycle_lengths[origin]
                
            if all(seen.values()):
                x = 1
                for cycle_length in cycle_lengths.values():
                    x = x * cycle_length // math.gcd(x, cycle_length)
                print(x)
                exit(0)
        
        if module.type == "%":
            if pulse == "lo":
                module.memory = "on" if module.memory == "off" else "off"
                outgoing = "hi" if module.memory == "on" else "lo"
                for x in module.outputs:
                    q.append((module.name, x, outgoing))
        else:
            module.memory[origin] = pulse
            outgoing = "lo" if all(x == "hi" for x in module.memory.values()) else "hi"
            for x in module.outputs:
                q.append((module.name, x, outgoing))

244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
244178746156661
24417874

KeyboardInterrupt: 

: 